In [1]:
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

In [2]:
resp = requests.get("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture")
soup = BeautifulSoup(resp.content, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})
rowsList = []
year = ""
for j in range(0, 10):
    table = tables[j]
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for i in range(1, len(rows)):
        cells = rows[i].find_all("td")
        if (len(cells) == 1):
            year = cells[0].text.split("(")[0].split("/")[0]
        if (len(cells) == 2):
            title = cells[0].find("a").text
            winner = False
            if rows[i].has_attr("style"):
                if rows[i]["style"].split(":")[1] == "#FAEB86":
                    winner = True
            producers = cells[1].text.replace("\n", "").replace("and ", "").split(",")

            rowsList.append({
                "Title": title,
                "producers":producers,
                "winner":winner,
                "year": year
            })
bestpicture = pd.DataFrame(rowsList)
bestpicture.head()    

,Title,producers,winner,year
0,Wings,[Paramount Famous Lasky],True,1927
1,The Racket,[The Caddo Company],False,1927
2,7th Heaven,[Fox],False,1927
3,The Broadway Melody,[Metro-Goldwyn-Mayer],True,1928
4,Alibi,[Feature Productions],False,1928


In [3]:
resp = requests.get("https://en.wikipedia.org/wiki/BAFTA_Award_for_Best_Film")
soup = BeautifulSoup(resp.content, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})
rowsList = []
year = ""
for table in tables:
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for i in range(1, len(rows)):
        cells = rows[i].find_all("td")
        if (len(cells) == 1):
            year = cells[0].text.split("(")[0].split("/")[0]
        else:
            if (len(cells) == 5):
                title = cells[1].find("a").text
                winner = True
            if (len(cells) == 4):
                title = cells[0].find("a").text
                winner = False
                
            rowsList.append({
                "Title": title,
                "BAFTA":winner,
                "year": year
            })
bafta = pd.DataFrame(rowsList)
bafta.head()

,BAFTA,Title,year
0,True,The Best Years of Our Lives,1947
1,True,Hamlet,1948
2,False,Crossfire,1948
3,False,The Fallen Idol,1948
4,False,Monsieur Vincent,1948


In [4]:
resp = requests.get("https://en.wikipedia.org/wiki/National_Board_of_Review_Award_for_Best_Film")
soup = BeautifulSoup(resp.content, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})
rowsList = []
year = ""
for table in tables:
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for i in range(1, len(rows)):
        cells = rows[i].find_all("td")
        title = cells[1].text.replace(" ‡", "").replace(" §", "").replace(" †", "")
        year = cells[0].text
                
        rowsList.append({
            "Title": title,
            "NBRA": True,
            "year": year
        })
nbra = pd.DataFrame(rowsList)

In [5]:
resp = requests.get("https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Motion_Picture_%E2%80%93_Drama")
soup = BeautifulSoup(resp.content, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})
rowsList = []
year = ""
for table in tables:
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for i in range(1, len(rows)):
        cells = rows[i].find_all("td")
        if (len(cells) == 4):
            year = cells[0].text
            title = cells[1].find("a").text
            winner = True
        if (len(cells) == 3):
            title = cells[0].find("a").text
            winner = False
                
        rowsList.append({
            "Title": title,
            "GOLDENGLOBE":winner,
            "year": year
        })
goldenglobe = pd.DataFrame(rowsList)

In [6]:
resp = requests.get("https://en.wikipedia.org/wiki/Satellite_Award_for_Best_Film#Drama_%281996%E2%80%932009,_2018%29")
soup = BeautifulSoup(resp.content, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})
rowsList = []
year = ""
for table in tables:
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for i in range(1, len(rows)):
        cells = rows[i].find_all("td")
        if (len(cells) == 1):
            year = cells[0].text.split("\n")[0]
        else:
            if (len(cells) == 3):
                year = cells[0].text.split("\n")[0]
                title = cells[1].find("a").text
                winner = True
            if (len(cells) == 2):
                title = cells[0].find("a").text
                winner = False

            rowsList.append({
                "Title": title,
                "SATELLITE": winner,
                "year": year
            })
satellite = pd.DataFrame(rowsList)


In [7]:
resp = requests.get("https://en.wikipedia.org/wiki/Directors_Guild_of_America_Award_for_Outstanding_Directing_%E2%80%93_Feature_Film")
soup = BeautifulSoup(resp.content, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})
rowsList = []
year = ""
for table in tables:
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for i in range(1, len(rows)):
        cells = rows[i].find_all("td")
        if (len(cells) == 4):
            year = cells[0].text.split("(")[0]
            title = cells[2].find("a").text
            winner = True
        if (len(cells) == 2):
            title = cells[1].find("a").text
            winner = False

        rowsList.append({
            "Title": title,
            "DGA": winner,
            "year": year
        })
dga = pd.DataFrame(rowsList)

In [49]:
movies = pd.read_csv("./movies.csv")
tags = pd.read_csv("./tags.csv")
movie_tags = movies.merge(tags, on=["movieId"])
movie_tags["Title"] = movie_tags.title.str.split(" ")
movie_tags["Title"] = movie_tags.Title.str[:-1]
movie_tags["Title"] = movie_tags["Title"].str.join(' ')
movie_tags = movie_tags.drop(["genres", "userId", "timestamp", "title"], axis=1)
movie_tags.head()

,movieId,tag,Title
0,1,pixar,Toy Story
1,1,pixar,Toy Story
2,1,fun,Toy Story
3,2,fantasy,Jumanji
4,2,magic board game,Jumanji


In [90]:
movie_tags_joined = pd.DataFrame(movie_tags.groupby(movie_tags['Title']).tag.apply(lambda a: ", ".join(a)))
movie_tags_joined = movie_tags_joined.reset_index()

In [52]:
dfs = []
for name in bestpicture["Title"]:
    name = name.replace(" ", "+")
    resp = requests.get("http://www.omdbapi.com/?t=" + name + "&plot=full&apikey=8de699a6")
    movie_json = resp.json()
    movie_df = json_normalize(movie_json)
    dfs.append(movie_df)

In [53]:
imdb = pd.concat(dfs, ignore_index=True, sort=True)

In [92]:
oscar_movies = pd.concat([bestpicture, imdb.drop(["Title"], axis=1)], axis=1)
oscar_movies = oscar_movies.drop(["DVD", "Error", "BoxOffice", "Poster", "Type", "Website", "totalSeasons"], axis=1)

oscar_movies = oscar_movies.merge(bafta.drop(["year"], axis=1), on=["Title"], how="left")
oscar_movies = oscar_movies.merge(nbra.drop(["year"], axis=1), on=["Title"], how="left")
oscar_movies = oscar_movies.merge(goldenglobe.drop(["year"], axis=1), on=["Title"], how="left")
oscar_movies = oscar_movies.merge(satellite.drop(["year"], axis=1), on=["Title"], how="left")
oscar_movies = oscar_movies.merge(dga.drop(["year"], axis=1), on=["Title"], how="left")
oscar_movies = oscar_movies.merge(movie_tags_joined, on=["Title"], how="left")
oscar_movies.BAFTA.fillna(False, inplace=True)
oscar_movies.NBRA.fillna(False, inplace=True)
oscar_movies.GOLDENGLOBE.fillna(False, inplace=True)
oscar_movies.SATELLITE.fillna(False, inplace=True)
oscar_movies.DGA.fillna(False, inplace=True)
oscar_movies.tag.fillna("none", inplace=True)
oscar_movies.to_csv(path_or_buf="./oscar_movies.csv", index=False)
#oscar_movies[oscar_movies.tag != "none"]
oscar_movies

,Title,producers,winner,year,Actors,Awards,Country,Director,Genre,Language,...,Year,imdbID,imdbRating,imdbVotes,BAFTA,NBRA,GOLDENGLOBE,SATELLITE,DGA,tag
0,Wings,[Paramount Famous Lasky],True,1927,"Clara Bow, Charles 'Buddy' Rogers, Richard Arl...",Won 2 Oscars. Another 3 wins & 1 nomination.,USA,"William A. Wellman, Harry d'Abbadie d'Arrast","Drama, Romance, War, Action",English,...,1927,tt0018578,7.7,"9,884",False,False,False,False,False,none
1,The Racket,[The Caddo Company],False,1927,"Robert Mitchum, Lizabeth Scott, Robert Ryan, W...",N/A,USA,"John Cromwell, Mel Ferrer, Tay Garnett, Nichol...","Crime, Drama, Film-Noir",English,...,1951,tt0043955,6.8,"2,056",False,False,False,False,False,none
2,7th Heaven,[Fox],False,1927,"Stephen Collins, Catherine Hicks, Beverley Mit...",Nominated for 1 Primetime Emmy. Another 20 win...,USA,N/A,"Drama, Family, Romance",English,...,1996–2007,tt0115083,5.2,"22,707",False,False,False,False,False,none
3,The Broadway Melody,[Metro-Goldwyn-Mayer],True,1928,"Charles King, Anita Page, Bessie Love",Won 1 Oscar. Another 2 nominations.,USA,Harry Beaumont,"Drama, Musical, Romance",English,...,1929,tt0019729,6.2,"5,601",False,False,False,False,False,none
4,Alibi,[Feature Productions],False,1928,"Achmed Akkabi, Georgina Verbaan, Frederik Brom...",1 win.,Netherlands,Johan Nijenhuis,Comedy,Dutch,...,2008,tt0827170,5.7,796,False,False,False,False,False,none
5,Hollywood Revue,[Metro-Goldwyn-Mayer],False,1928,"Conrad Nagel, Jack Benny, John Gilbert, Norma ...",N/A,USA,"Charles Reisner, Christy Cabanne",Musical,English,...,1929,tt0019993,7.4,"1,586",False,False,False,False,False,none
6,In Old Arizona,[Fox],False,1928,"Warner Baxter, Edmund Lowe, Dorothy Burgess",Won 1 Oscar. Another 4 nominations.,USA,Irving Cummings,"Romance, Western","English, Spanish, Italian",...,1928,tt0020018,5.7,668,False,False,False,False,False,none
7,The Patriot,[Paramount Famous Lasky],False,1928,"Mel Gibson, Heath Ledger, Joely Richardson, Ja...",Nominated for 3 Oscars. Another 8 wins & 18 no...,"USA, Germany",Roland Emmerich,"Action, Drama, History, War",English,...,2000,tt0187393,7.2,"234,528",False,False,False,False,False,none
8,All Quiet on the Western Front,[Universal],True,1929,"Louis Wolheim, Lew Ayres, John Wray, Arnold Lucy",Won 2 Oscars. Another 5 wins & 2 nominations.,USA,Lewis Milestone,"Drama, War","English, French, German, Latin",...,1930,tt0020629,8.1,"52,868",False,False,False,False,False,none
9,The Big House,[Cosmopolitan],False,1929,"Chester Morris, Wallace Beery, Lewis Stone, Ro...",Won 2 Oscars. Another 2 nominations.,USA,"George W. Hill, Ward Wing","Crime, Drama, Thriller","English, Russian",...,1930,tt0020686,7.2,"1,589",False,False,False,False,False,none
